<a href="https://colab.research.google.com/github/Ana-AlonsoCanizares/AA_GRUPO3/blob/main/Limpieza_AT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Faltan los datos de Guadalajara de AT**


In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 2.7 MB/s eta 0:00:00


In [3]:
import os
import pandas as pd
import numpy as np
from unidecode import unidecode
import matplotlib.pyplot as plt

In [4]:
# # Asegúrarse de cambiar la ruta por el nombre real de la carpeta en Google Drive
# folder_path = '/content/drive/My Drive/PFG_FASPAS/SAAF'
# docs_xlsx = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]
# print(docs_xlsx)

# # Asegúrarse de cambiar la ruta por el nombre real de la carpeta en Google Drive
# folder_path_csv = '/content/drive/My Drive/PFG_FASPAS/Zonas_Prioritarias'
# docs_csv = [f for f in os.listdir(folder_path_csv) if f.endswith('.csv')]
# print(docs_csv)

In [5]:
# dic_dataframes = {}
# dic_zonas = {}

# for doc in docs_xlsx:
#     entire_path = os.path.join(folder_path, doc)
#     df = pd.read_excel(entire_path)
#     # Uso el nombre del archivo como clave
#     dic_dataframes[doc] = df

# for doc in docs_csv:
#   entire_path = os.path.join(folder_path_csv, doc)
#   df = pd.read_csv(entire_path)
#   # Uso el nombre del archivo como clave
#   dic_zonas[doc] = df

# # Imprimir todas las claves
# for clave in dic_dataframes.keys():
#     print(clave)

In [6]:
# CU_SAAF = dic_dataframes.get("CU_SAAF.xlsx")
# CU_AT = dic_dataframes.get("CU_AT.xlsx")
# CU_EMPLEO = dic_dataframes.get("CU_EMPLEO.xlsx")
# CU_LOG = dic_dataframes.get("CU_LOG.xlsx")
# # CU_SOCIOS = dic_dataframes.get("CU_SOCIOS.xlsx")

In [111]:
# CÓDIGO SOLO PARA EL CURRO (PQ NO PUEDO ACCEDER AL DRIVE)
# Primero cogemos todos los trimestres de CU para probar
CU_Trim1_2022_AT = pd.read_excel('/content/CU_Trim1_2022_AT.xlsx')
CU_Trim2_2022_AT = pd.read_excel('/content/CU_Trim2_2022_AT.xlsx')
CU_Trim3_2022_AT = pd.read_excel('/content/CU_Trim3_2022_AT.xlsx')
CU_Trim4_2022_AT = pd.read_excel('/content/CU_Trim4_2022_AT.xlsx')
CU_Trim1_2023_AT = pd.read_excel('/content/CU_Trim1_2023_AT.xlsx')
CU_Trim2_2023_AT = pd.read_excel('/content/CU_Trim2_2023_AT.xlsx')
CU_Trim3_2023_AT = pd.read_excel('/content/CU_Trim3_2023_AT.xlsx')
CU_Trim4_2023_AT = pd.read_excel('/content/CU_Trim4_2023_AT.xlsx')

In [110]:
# Definir la función que realiza las transformaciones
def transform_df(df):
    col_excluded = ['FECHA NACIMIENTO', 'FECHA DE NACIMIENTO', 'FECHA NAC']

    # Convertir todas las columnas de tipo object a mayúsculas, excepto las especificadas
    for column in df.columns:
        if df[column].dtype == 'object' and column not in col_excluded:
            df[column] = df[column].apply(lambda x: unidecode(x.upper()) if isinstance(x, str) else x)

    # Ahora, aplicamos unidecode a los nombres de las columnas
    df.columns = [unidecode(col.upper()) for col in df.columns]

    return df

def add_trim(df, trim):
    df['YEAR - TRIMESTRE'] = trim
    return df

def kept_columns(df):
  # Lista de columnas a quedarse
  all_needed_columns = ['FECHA NACIMIENTO (DD/MM/AAAA)', 'SEXO\n(DESPLEGABLE)', 'CODIGO POSTAL', 'PROVINCIA\n(DESPLEGABLE)', 'LOCALIDAD', 'MOTIVO DE DERIVACION', 'GRADO DISCAPACIDAD\n(INDICAR PORCENTAJE O DEJAR EN BLANCO)', 'YEAR - TRIMESTRE']
  df = df[df.columns.intersection(all_needed_columns)]
  return df

def mapping_names(df):
    # Diccionario con los mapeos deseados
    columns_map = {
        'SEXO\n(DESPLEGABLE)': 'GENERO',
        'FECHA NACIMIENTO (DD/MM/AAAA)': 'FECHA NACIMIENTO',
        'CODIGO POSTAL': 'CP',
        'MOTIVO DE DERIVACION': 'MOTIVO DERIVACION',
        'PROVINCIA\n(DESPLEGABLE)': 'PROVINCIA',
        'GRADO DISCAPACIDAD\n(INDICAR PORCENTAJE O DEJAR EN BLANCO)': 'GRADO DISCAPACIDAD'
    }

    # Crear un nuevo diccionario para los nombres de columnas
    rename_columns = {}

    # Iterar sobre las columnas y aplicar el mapeo
    for col in df.columns:
        norm_col = columns_map.get(col, col)
        rename_columns[col] = norm_col

    # Renombrar las columnas del DataFrame
    df.rename(columns=rename_columns, inplace=True)
    return df

def CP_loc(df):

  df['CP'] =df['CP'].astype('Int64')

  df['YEAR NACIMIENTO'] = df['FECHA NACIMIENTO'].dt.year

  df = df.dropna(subset=['CP', 'LOCALIDAD'], how='all')

  CP_loc_filter = df['CP'].isnull() & df['LOCALIDAD'].isnull()

  df = df[~CP_loc_filter]
  return df

def entire_df(df, columns):
  # Eliminamos duplicados en las columnas de identificación
  df.drop_duplicates(subset=columns, inplace=True)

  # Agrupamos por las columnas de identificación personales y contamos las apariciones por trimestre
  grouped = df.groupby(['FECHA NACIMIENTO', 'GENERO', 'CP', 'PROVINCIA', 'LOCALIDAD', 'MOTIVO DERIVACION', 'GRADO DISCAPACIDAD', 'YEAR NACIMIENTO'])

  # Creamos un DataFrame con la cantidad de apariciones por trimestre
  df_return = grouped['YEAR - TRIMESTRE'].value_counts().unstack(fill_value=0).reset_index()

  # Cambiamos los nombres de las columnas para que sean más específicos
  new_column_names = {trimestre: f'{trimestre.replace(" ", "_")}' for trimestre in df_return.columns if 'TRIM' in trimestre}
  df_return.rename(columns=new_column_names, inplace=True)
  return df_return

In [112]:
CU_Trim1_2022_AT = transform_df(CU_Trim1_2022_AT)
CU_Trim1_2022_AT = add_trim(CU_Trim1_2022_AT, '2022 TRIM 1')
CU_Trim1_2022_AT = kept_columns(CU_Trim1_2022_AT)
CU_Trim1_2022_AT = mapping_names(CU_Trim1_2022_AT)
CU_Trim1_2022_AT = CP_loc(CU_Trim1_2022_AT)

CU_Trim2_2022_AT = transform_df(CU_Trim2_2022_AT)
CU_Trim2_2022_AT = add_trim(CU_Trim2_2022_AT, '2022 TRIM 2')
CU_Trim2_2022_AT = kept_columns(CU_Trim2_2022_AT)
CU_Trim2_2022_AT = mapping_names(CU_Trim2_2022_AT)
CU_Trim2_2022_AT = CP_loc(CU_Trim2_2022_AT)

CU_Trim3_2022_AT = transform_df(CU_Trim3_2022_AT)
CU_Trim3_2022_AT = add_trim(CU_Trim3_2022_AT, '2022 TRIM 3')
CU_Trim3_2022_AT = kept_columns(CU_Trim3_2022_AT)
CU_Trim3_2022_AT = mapping_names(CU_Trim3_2022_AT)
CU_Trim3_2022_AT = CP_loc(CU_Trim3_2022_AT)

CU_Trim4_2022_AT = transform_df(CU_Trim4_2022_AT)
CU_Trim4_2022_AT = add_trim(CU_Trim4_2022_AT, '2022 TRIM 4')
CU_Trim4_2022_AT = kept_columns(CU_Trim4_2022_AT)
CU_Trim4_2022_AT = mapping_names(CU_Trim4_2022_AT)
CU_Trim4_2022_AT = CP_loc(CU_Trim4_2022_AT)

CU_Trim1_2023_AT = transform_df(CU_Trim1_2023_AT)
CU_Trim1_2023_AT = add_trim(CU_Trim1_2023_AT, '2023 TRIM 1')
CU_Trim1_2023_AT = kept_columns(CU_Trim1_2023_AT)
CU_Trim1_2023_AT = mapping_names(CU_Trim1_2023_AT)
CU_Trim1_2023_AT = CP_loc(CU_Trim1_2023_AT)

CU_Trim2_2023_AT = transform_df(CU_Trim2_2023_AT)
CU_Trim2_2023_AT = add_trim(CU_Trim2_2023_AT, '2023 TRIM 2')
CU_Trim2_2023_AT = kept_columns(CU_Trim2_2023_AT)
CU_Trim2_2023_AT = mapping_names(CU_Trim2_2023_AT)
CU_Trim2_2023_AT = CP_loc(CU_Trim2_2023_AT)

CU_Trim3_2023_AT = transform_df(CU_Trim3_2023_AT)
CU_Trim3_2023_AT = add_trim(CU_Trim3_2023_AT, '2023 TRIM 3')
CU_Trim3_2023_AT = kept_columns(CU_Trim3_2023_AT)
CU_Trim3_2023_AT = mapping_names(CU_Trim3_2023_AT)
CU_Trim3_2023_AT = CP_loc(CU_Trim3_2023_AT)

CU_Trim4_2023_AT = transform_df(CU_Trim4_2023_AT)
CU_Trim4_2023_AT = add_trim(CU_Trim4_2023_AT, '2023 TRIM 4')
CU_Trim4_2023_AT = kept_columns(CU_Trim4_2023_AT)
CU_Trim4_2023_AT = mapping_names(CU_Trim4_2023_AT)
CU_Trim4_2023_AT = CP_loc(CU_Trim4_2023_AT)

# Concatenamos todos los DataFrames (Asegúrate de que todos tienen las mismas columnas)
CU_df_total = pd.concat([CU_Trim1_2022_AT, CU_Trim2_2022_AT, CU_Trim3_2022_AT, CU_Trim4_2022_AT, CU_Trim1_2023_AT, CU_Trim2_2023_AT, CU_Trim3_2023_AT, CU_Trim4_2023_AT])
all_needed_columns = ['FECHA NACIMIENTO', 'GENERO', 'CP', 'PROVINCIA', 'LOCALIDAD', 'MOTIVO DERIVACION', 'GRADO DISCAPACIDAD', 'YEAR - TRIMESTRE', 'YEAR NACIMIENTO']
CU_AT = entire_df(CU_df_total, all_needed_columns)

<ipython-input-110-ab05004eec47>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns=rename_columns, inplace=True)
<ipython-input-110-ab05004eec47>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CP'] =df['CP'].astype('Int64')
<ipython-input-110-ab05004eec47>:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-